Please run those two cells before running the Notebook!

As those plotting settings are standard throughout the book, we do not show them in the book every time we plot something.

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
# FIX: Use the official public API path from pandas.errors
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# feel free to modify, for example, change the context to "notebook"
sns.set_theme(context="talk", style="whitegrid", 
              palette="colorblind", color_codes=True, 
              rc={"figure.figsize": [12, 8]})

# Chapter 9 - Modeling Volatility with GARCH class models

## 9.1 Modeling stock returns' volatility with ARCH models

### How to do it...

1. Import the libraries:

In [ ]:
# FIX: Install the arch library for volatility modeling
!pip install arch

In [ ]:
import pandas as pd
import yfinance as yf
from arch import arch_model

2. Specify the risky asset and the time horizon:

In [ ]:
RISKY_ASSET = "GOOG"
START_DATE = "2015-01-01"
END_DATE = "2021-12-31"

3. Download data from Yahoo Finance:

In [ ]:
df = yf.download(RISKY_ASSET,
                 start=START_DATE,
                 end=END_DATE,
                 auto_adjust=True) # FIX: Replaced 'adjusted' with 'auto_adjust'

print(f"Downloaded {df.shape[0]} rows of data.")

4. Calculate daily returns:

In [ ]:
returns = 100 * df["Adj Close"].pct_change().dropna()
returns.name = "asset_returns"
print(f"Average return: {round(returns.mean(), 2)}%")
returns.plot(title=f"{RISKY_ASSET} returns: {START_DATE} - {END_DATE}");

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_1", dpi=200)

5. Specify the ARCH model:

In [ ]:
model = arch_model(returns, mean="Zero", vol="ARCH", p=1, q=0)

6. Estimate the model and print the summary:

In [ ]:
fitted_model = model.fit(disp="off")
print(fitted_model.summary())

7. Plot the residuals and the conditional volatility:

In [ ]:
fitted_model.plot(annualize="D")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_2", dpi=200)

In the cell below we confirm that the standardized residuals are simply residuals divided by the conditional volatility.

In [ ]:
diagnostics_dict = {
    "resids": fitted_model.resid,
    "conditional_volatility": fitted_model.conditional_volatility,
    "std_resid": fitted_model.std_resid,
    "std_resid_manual": fitted_model.resid / fitted_model.conditional_volatility,
}

df_diagnostics = pd.DataFrame(data = diagnostics_dict)
df_diagnostics

### There's more

Test the residuals of the ARCH(1) model with the LM test.

In [ ]:
from statsmodels.stats.diagnostic import het_arch
het_arch(fitted_model.resid)

As the residuals come from a model in which we estimated two parameters (omega and alpha), we should correct for that when using the `het_arch` test.

In [ ]:
het_arch(fitted_model.resid, ddof=2)

## 9.2 Modeling stock returns' volatility with GARCH models

### How to do it...

1. Specify the GARCH model:

In [ ]:
model = arch_model(returns, mean="Zero", vol="GARCH", p=1, q=1)

2. Estimate the model and print the summary:

In [ ]:
fitted_model = model.fit(disp="off")
print(fitted_model.summary())

3. Plot the residuals and the conditional volatility:

In [ ]:
fitted_model.plot(annualize="D")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_3", dpi=200)

## 9.3 Forecasting volatility using GARCH models

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from arch import arch_model

2. Download data from Yahoo Finance and calculate simple returns:

In [ ]:
df = yf.download("MSFT",
                 start="2015-01-01",
                 end="2021-12-31",
                 adjusted=True)

returns = 100 * df["Adj Close"].pct_change().dropna()
returns.name = "asset_returns"

3. Specify the GARCH model:

In [ ]:
model = arch_model(returns, mean="Zero", vol="GARCH", dist="t",
                   p=1, q=1)

4. Define the split date and fit the model:

In [ ]:
SPLIT_DATE = datetime(2021, 1, 1)
fitted_model = model.fit(last_obs=SPLIT_DATE, disp="off")

5. Create and inspect the analytical forecasts:

In [ ]:
forecasts_analytical = fitted_model.forecast(horizon=3, 
                                             start=SPLIT_DATE,
                                             reindex=False)
forecasts_analytical.variance.plot(
    title="Analytical forecasts for different horizons"
)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_4", dpi=200)

In [ ]:
forecasts_analytical.variance

6. Create and inspect the simulation forecasts:

In [ ]:
forecasts_simulation = fitted_model.forecast(horizon=3, 
                                             start=SPLIT_DATE,
                                             method="simulation",
                                             reindex=False)
forecasts_simulation.variance.plot(
    title="Simulation forecasts for different horizons"
)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_6", dpi=200)

In [ ]:
forecasts_simulation.variance

7. Create and inspect the bootstrap forecasts:

In [ ]:
forecasts_bootstrap = fitted_model.forecast(horizon=3, 
                                            start=SPLIT_DATE,
                                            method="bootstrap",
                                            reindex=False)
forecasts_bootstrap.variance.plot(
    title="Bootstrap forecasts for different horizons"
)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_7", dpi=200)

In [ ]:
forecasts_bootstrap.variance

### There's more

1. Import the libraries:

In [ ]:
import numpy as np

2. Estimate the volatility forecasts for 2020 using the analytic and bootstrap approaches:

In [ ]:
# define the forecast horizon
FCST_HORIZON = 10

vol_analytic = (
    fitted_model.forecast(horizon=FCST_HORIZON, 
                          start=datetime(2020, 1, 1),
                          reindex=False)
    .residual_variance["2020"]
    .apply(np.sqrt)
)

vol_bootstrap = (
    fitted_model.forecast(horizon=FCST_HORIZON, 
                          start=datetime(2020, 1, 1),
                          method="bootstrap",
                          reindex=False)
    .residual_variance["2020"]
    .apply(np.sqrt)
)


3. Get the conditional volatility for 2020:

In [ ]:
vol = fitted_model.conditional_volatility["2020"]

4. Create the hedgehog plot:

In [ ]:
ax = vol.plot(
    title="Comparison of analytical vs bootstrap volatility forecasts",
    alpha=0.5
)
ind = vol.index

for i in range(0, 240, 10):
    vol_a = vol_analytic.iloc[i]
    vol_b = vol_bootstrap.iloc[i]
    start_loc = ind.get_loc(vol_a.name)
    new_ind = ind[(start_loc+1):(start_loc+FCST_HORIZON+1)]
    vol_a.index = new_ind
    vol_b.index = new_ind
    ax.plot(vol_a, color="r")
    ax.plot(vol_b, color="g")

labels = ["Volatility", "Analytical Forecast", "Bootstrap Forecast"]
legend = ax.legend(labels)

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_8", dpi=200)

## 9.4 Multivariate volatility forecasting with the CCC-GARCH model

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
from arch import arch_model

2. Specify the risky asset and the time horizon:

In [ ]:
RISKY_ASSETS = ["GOOG", "MSFT", "AAPL"]
START_DATE = "2015-01-01"
END_DATE = "2021-12-31"

3. Download data from Yahoo Finance:

In [ ]:
df = yf.download(RISKY_ASSETS,
                 start=START_DATE,
                 end=END_DATE,
                 adjusted=True)

print(f"Downloaded {df.shape[0]} rows of data.")

4. Calculate daily returns:

In [ ]:
returns = 100 * df["Adj Close"].pct_change().dropna()
returns.plot(subplots=True, 
             title=f"Stock returns: {START_DATE} - {END_DATE}")

sns.despine()
plt.tight_layout()
# plt.savefig("images/figure_9_9", dpi=200)

5. Define lists for storing objects:

In [ ]:
coeffs = []
cond_vol = []
std_resids = []
models = []

6. Estimate the univariate GARCH models:

In [ ]:
for asset in returns.columns:
    # specify and fit the model
    model = arch_model(returns[asset], mean="Constant", 
                       vol="GARCH", p=1, q=1)
    model = model.fit(update_freq=0, disp="off");
    
    # store results in the lists 
    coeffs.append(model.params)
    cond_vol.append(model.conditional_volatility)
    std_resids.append(model.std_resid)
    models.append(model)

7. Store the results in DataFrames:

In [ ]:
coeffs_df = pd.DataFrame(coeffs, index=returns.columns)
cond_vol_df = (
    pd.DataFrame(cond_vol)
    .transpose()
    .set_axis(returns.columns,
              axis="columns")
)
std_resids_df = (
    pd.DataFrame(std_resids)
    .transpose()
    .set_axis(returns.columns, 
              axis="columns")
)

In [ ]:
coeffs_df

8. Calculate the constant conditional correlation matrix (R):

In [ ]:
R = (
    std_resids_df
    .transpose()
    .dot(std_resids_df)
    .div(len(std_resids_df))
)

9. Calculate the 1-step ahead forecast of the conditional covariance matrix :

In [ ]:
# define objects
diag = []
D = np.zeros((len(RISKY_ASSETS), len(RISKY_ASSETS)))

# populate the list with conditional variances
for model in models:
    diag.append(model.forecast(horizon=1).variance.iloc[-1, 0])
# take the square root to obtain volatility from variance
diag = np.sqrt(diag)
# fill the diagonal of D with values from diag
np.fill_diagonal(D, diag)

# calculate the conditional covariance matrix
H = np.matmul(np.matmul(D, R.values), D)

In [ ]:
H

## 9.5 Forecasting the conditional covariance matrix using DCC-GARCH

### Getting ready

Before executing the following code, please make sure to run the code from the previous recipe to have the data available. 

### How to do it...

1. Import the libraries:

In [ ]:
import pandas as pd

Setup the connection between Python and R using `rpy2`:

In [ ]:
%load_ext rpy2.ipython

2. Install `rmgarch` R package and load it:

In [ ]:
%%R

install.packages("rmgarch", repos = "http://cran.us.r-project.org")
library(rmgarch)

3. Import the dataset into R:

In [ ]:
%%R -i returns
print(head(returns, 5))

4. Define the model specification:

In [ ]:
%%R

# define GARCH(1,1) model
univariate_spec <- ugarchspec(
    mean.model = list(armaOrder = c(0,0)),
    variance.model = list(garchOrder = c(1,1), 
                          model = "sGARCH"),
    distribution.model = "norm"
)

# define DCC(1,1) model
n <- dim(returns)[2]
dcc_spec <- dccspec(
    uspec = multispec(replicate(n, univariate_spec)),
    dccOrder = c(1,1),
    distribution = "mvnorm"
)

dcc_spec

5. Estimate the model:

In [ ]:
%%R
dcc_fit <- dccfit(dcc_spec, data=returns)
dcc_fit

6. Calculate the 5-step ahead forecasts:

In [ ]:
%%R
forecasts <- dccforecast(dcc_fit, n.ahead = 5)

7. Access the forecasts:

In [ ]:
%%R

# conditional covariance matrix
forecasts@mforecast$H
# conditional correlation matrix
forecasts@mforecast$R
# proxy correlation process
forecasts@mforecast$Q
# conditional mean forecasts
forecasts@mforecast$mu

### There's more

In [ ]:
%%R

# parallelized DCC-GARCH(1,1)

library("parallel")

# set up the cluster
cl <- makePSOCKcluster(3)

# define parallelizable specification
parallel_fit <- multifit(multispec(replicate(n, univariate_spec)), 
                         returns, 
                         cluster = cl)

# fit the DCC-GARCH model
dcc_fit <- dccfit(dcc_spec, 
                  data = returns, 
                  fit.control = list(eval.se = TRUE), 
                  fit = parallel_fit, 
                  cluster = cl)

# stop the cluster
stopCluster(cl)

In [ ]:
%%R

dcc_fit